# Resolução da lista 3 de NLP

## Alunos:
  - Eduardo Brasil Araujo
  - Gideão Pinheiro"

# Questão 1

In [ ]:
!pip install transformers

In [1]:
import transformers as ppb
import pandas as pd
import numpy as np
import random
import torch

e:\localhost\Desktop\dor-repos\computer-science\natural-language-processing\repo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

distil_config = ppb.DistilBertConfig(
    max_position_embeddings=1024 * 8
)

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, config=distil_config, ignore_mismatched_sizes=True)
model = model_class.from_pretrained(pretrained_weights, config=distil_config, ignore_mismatched_sizes=True)

Some weights of DistilBertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized because the shapes did not match:
- distilbert.embeddings.position_embeddings.weight: found shape torch.Size([512, 768]) in the checkpoint and torch.Size([8192, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
df = pd.read_csv('../datasets/preprocessed_website_classification.csv')

In [6]:
df.dropna(inplace=True)

In [7]:
data = df.cleaned_website_text
data = data.tolist()

In [8]:
random.seed(124)
random.shuffle(data)

In [9]:
batch_1 = data[:12]
batch_1 = pd.DataFrame(batch_1)

In [10]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (1824 > 512). Running this sequence through the model will result in indexing errors


In [11]:
print(tokenized)

0     [101, 4596, 18064, 19413, 5332, 7216, 2833, 28...
1     [101, 2057, 22499, 2213, 3229, 2953, 3795, 205...
2     [101, 21025, 2213, 2033, 3972, 28775, 2833, 21...
3     [101, 3419, 6738, 9982, 3063, 3419, 6738, 4012...
4     [101, 3296, 9353, 2213, 25583, 17899, 5860, 13...
5     [101, 11834, 2282, 11834, 2208, 22128, 11834, ...
6     [101, 8694, 2378, 2489, 4274, 2557, 2444, 2739...
7     [101, 18072, 5003, 4168, 5239, 5003, 4168, 938...
8     [101, 5460, 2444, 2678, 2006, 4115, 6819, 2624...
9     [101, 2535, 2377, 1054, 2361, 11834, 9954, 253...
10    [101, 6745, 4368, 2739, 4533, 2739, 2444, 2998...
11    [101, 2522, 28748, 4372, 24269, 2522, 28748, 4...
Name: 0, dtype: object


In [12]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [13]:
np.array(padded).shape

(12, 4800)

In [14]:
print(padded)

[[  101  4596 18064 ...     0     0     0]
 [  101  2057 22499 ...     0     0     0]
 [  101 21025  2213 ...     0     0     0]
 ...
 [  101  2535  2377 ...     0     0     0]
 [  101  6745  4368 ...  2121  2615   102]
 [  101  2522 28748 ...     0     0     0]]


In [179]:
def remove_elements_to_match_count(arr, n):
    arr = arr.tolist()
    
    unique_elements, counts = np.unique(arr, return_counts=True)
    max_size_array = max(len(line) for line in arr)
    
    
    while max_size_array > n:
        element_to_remove = unique_elements[np.argmin(counts)]
        
        for i, line in enumerate(arr):
            indices_to_remove = np.where(np.array(line) == element_to_remove)
            arr[i] = np.delete(arr[i], indices_to_remove).tolist()

        max_size_array = max(len(line) for line in arr)
        for i, line in enumerate(arr):
            line_length = len(line)
            if line_length < max_size_array:
                arr[i] += [0] * (max_size_array - line_length)

        unique_elements, counts = np.unique(arr, return_counts=True)
        
        exclude_index = np.argmin(unique_elements)
        
        unique_elements = np.delete(unique_elements, exclude_index)
        counts = np.delete(counts, exclude_index)
        
    
    return np.array(arr)

new_padded = remove_elements_to_match_count(padded, 512)
# new_padded = remove_elements_to_match_count(np.array(normal), 2)
print(new_padded)

ValueError: attempt to get argmin of an empty sequence

In [75]:
def get_cut_value(orig, counted_freq, threshold):
    new_mat = orig.tolist()
    min_value = min(counted_freq)
    max_value = max(counted_freq)
    for i in range(min_value, max_value):
        up_to_max = 0
        for value in new_mat:
            new_arr = np.array(value)
            new_value = new_arr[np.isin(new_arr, np.where(counted_freq >= i)[0])]
            if len(new_value) > up_to_max:
                up_to_max = len(new_value)
        
        if up_to_max <= threshold:
            return counted_freq[counted_freq > i]

padded_shape = padded.shape
counts = np.bincount(padded.flatten())
padded[np.isin(padded.flatten(), get_cut_value(padded, counts, 512)).reshape(padded_shape)] = 0

KeyboardInterrupt: 

In [57]:
new_padded = padded.tolist()

In [58]:
to_padd = 512 - (len(new_padded[0]) % 512)
new_mat = []
print(to_padd, len(new_padded[0]))
for i, line in enumerate(new_padded):
    to_replace = np.array(line)
    to_replace = to_replace[to_replace != 0]
    print(len(to_replace))
    new_mat.append(to_replace + [0] * (512 - len(to_replace)))

122 12166
951


ValueError: operands could not be broadcast together with shapes (951,) (0,) 

In [35]:
padded = np.array(new_padded)

In [15]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(12, 4800)

In [16]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

# Question 2

In [1]:
from bertopic import BERTopic

e:\localhost\Desktop\dor-repos\computer-science\natural-language-processing\repo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Question 3

Abaixo está a criação do prompt utilizando o chat-GPT na versão 3.5
A criação do prompt segue o que foi indicado [neste artigo](https://arxiv.org/abs/2304.10428)
Na primeira etapa utilizamos a frase "I am an excelent linguist" para que as respostas sejam dadas utilizando os conhecimentos de linguística. 
Ainda nesta etapa informamos qual a tarefa queremos que seja realizada e informamos que as entidades são do tipo "animal" e após isso indicamos onde estão os exemplos.

Na segunda etapa damos alguns exemplos, fornecendo inputs parecidos com os que ele receberia e outputs que mostram o padrão da resposta.

Após isso, pedimos para que ele complete os outputs e passamos 5 inputs para que ele identifique as categorias.

![GPT-NER](../assets/2023-10-08_11-25.png)

O resultado é mostrado abaixo:

![Resultado](../assets/2023-10-08_11-25_1.png)

# Question 4